In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

import stan

import nest_asyncio
nest_asyncio.apply()
del nest_asyncio


OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [2]:
schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}


In [3]:
posterior = stan.build(schools_code, data=schools_data, random_seed=1)


Building...

In file included from /usr/local/lib/python3.9/site-packages/httpstan/include/stan/model/model_header.hpp:7,
                 from /root/.cache/httpstan/4.7.2/models/lfzkgibo/model_lfzkgibo.cpp:2:
/usr/local/lib/python3.9/site-packages/httpstan/include/stan/io/dump.hpp: In member function ‘virtual std::vector<std::complex<double> > stan::io::dump::vals_c(const string&) const’:
/usr/local/lib/python3.9/site-packages/httpstan/include/stan/io/dump.hpp:694:52: warning: comparison of integer expressions of different signedness: ‘int’ and ‘std::vector<double, std::allocator<double> >::size_type’ {aka ‘long unsigned int’} [-Wsign-compare]
  694 |       for (comp_iter = 0, real_iter = 0; real_iter < val_r->second.first.size();
      |                                          ~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~
/usr/local/lib/python3.9/site-packages/httpstan/include/stan/io/dump.hpp:707:24: warning: comparison of integer expressions of different signedness: ‘int’ and ‘std::vector<int>::size_


Building: 30.3s, done.Messages from stanc:
Warning in '/tmp/httpstan_ykwshjnr/model_lfzkgibo.stan', line 4, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_ykwshjnr/model_lfzkgibo.stan', line 5, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc


In [4]:
fit = posterior.sample(num_chains=4, num_samples=1000)


Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 3.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.36 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.19 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.18 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.19 seconds.
  Adjust your expectations accordingly!


In [5]:
fit["eta"]


array([[ 0.06569064,  1.07555266, -0.55215545, ...,  1.41114167,
        -0.55744645, -0.86580196],
       [ 0.38732515,  0.84298643, -0.11625492, ..., -0.85416625,
         1.94558782, -1.09278351],
       [-0.70225196,  0.51324704, -1.59100185, ..., -2.4886128 ,
         1.10105798,  0.45169692],
       ...,
       [ 0.2013854 ,  0.83863796, -1.11845423, ...,  0.70655335,
         0.12869399,  0.29676855],
       [ 0.75224962,  0.27064642,  1.07229314, ...,  0.05187624,
         0.52653385,  0.0886355 ],
       [-0.21515163,  1.17637997, -0.96446302, ..., -0.40114063,
         0.90874234,  1.02801608]])